### Import library

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os.path
import re

### Define some function

In [2]:
def scroll_down(driver):
    SCROLL_PAUSE_TIME = 1
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue
    return driver

def scroll_down_and_wait(driver):
    actions = ActionChains(driver)
    for _ in range(5):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(5)
    return driver

def parse_info_product(driver, item, label):
    driver.get(item)
    time.sleep(5)
    driver = scroll_down_and_wait(driver)
### This part for reload shop info
    if len(driver.find_elements_by_class_name("TuJk3S"))==0:
        print('not found info shop')
        driver.refresh()
        time.sleep(3)
        driver = scroll_down_and_wait(driver)
### This part for confirm 18+ if any   
    try:
        driver.find_element_by_xpath("//button[@class='btn btn-solid-primary btn--m btn--inline shopee-alert-popup__btn']").click()
        print('Confirmed 18+')
    except NoSuchElementException:
        pass
### This part for get item name
    info_dict = {}
    product_span = driver.find_elements_by_class_name("qaNIZv")
    if len(product_span)>0:
        info_dict.update({'Tên sp:': product_span[0].text.split('\n')[-1],'label:': label})
        ### This part for get item description
        for attr in driver.find_elements_by_class_name("kIo6pj"):
            info_dict.update({attr.text.split('\n')[0]:'-'.join(attr.text.split('\n')[1:])})
        ### This part for get price - discount - price discount
        if len(driver.find_elements_by_class_name("_3_ISdg"))==0:
            info_dict.update({'sale_price':driver.find_elements_by_class_name("_3n5NQx")[0].text})
            info_dict.update({'org_price':driver.find_elements_by_class_name("_3n5NQx")[0].text})
            info_dict.update({'discount':0})
        else:
            info_dict.update({'org_price':driver.find_elements_by_class_name("_3_ISdg")[0].text})
            info_dict.update({'sale_price':driver.find_elements_by_class_name("_3n5NQx")[0].text})
            info_dict.update({'discount':driver.find_elements_by_class_name("MITExd")[0].text})
        ### This part for get rating-star-selled
        if len(driver.find_elements_by_class_name("_3Oj5_n"))==0:
            info_dict.update({'star':0})
            info_dict.update({'rating':0})
        else:
            for rat in driver.find_elements_by_class_name("_3Oj5_n"):
                info_dict.update({'star':driver.find_elements_by_class_name("_3Oj5_n")[0].text})
                info_dict.update({'rating':driver.find_elements_by_class_name("_3Oj5_n")[1].text})
        try:
            my_property = driver.find_element_by_class_name('_3ZDC1p').find_element_by_xpath("//div[@class='_2JMB9h _3XaILN']").value_of_css_property("background-image")
            info_dict.update({'img_url': re.split('[()]',my_property)[1]})
        except NoSuchElementException:
            info_dict.update({'img_url': ''})
        info_dict.update({'selled':driver.find_elements_by_class_name("_22sp0A")[0].text})
        ### This part for get info about store
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[0]:driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[1]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[2]:driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[3]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[0]:driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[1]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[2]:driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[3]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[0]:driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[1]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[2]:driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[3]})
    else:
        pass
    return driver, info_dict


def get_link_product(driver):
    all_items = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
    all_urls = []
    for item in all_items:
        url = item.get_attribute('href')
        all_urls.append(url)
    return driver, all_urls

### Please change param at cell below

In [3]:
index = -10
filename = 'bach_hoa_online'
limit_prod = 601

### This part for execute to crawl data, please run each cell and wait 3s before continue

In [4]:
driver = webdriver.Firefox()
driver.get("https://shopee.vn/")
time.sleep(3)

In [5]:
popup_close_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'shopee-popup__close-btn')]")))
if popup_close_button:
    popup_close_button.click()

In [6]:
driver.maximize_window()

In [7]:
url_item = []
category = driver.find_elements_by_class_name('home-category-list__group')
for sub_item in category:
    for ssitem in sub_item.find_elements_by_class_name('home-category-list__category-grid'):
        url_item.append(ssitem.get_attribute('href'))
url_item = url_item[:-1]

In [8]:
label_marker = ['Thoi_trang','Thoi_trang','Dien_thoai_&_Phu_kien','Me_&_be','Thiet_bi_DT','Nha_cua_va_doi_song','May_tinh_&_Laptop','Suc_khoe_&_Sac_dep','May_anh_quay_phim',
               'Giay_dep_nu','Dong_ho','Giay_dep_nam','Phu_kien_thoi_trang','Thiet_bi_gia_dung','Bach_hoa_online','The_thao_du_lich','Voucher_dich_vu','Oto_xe_may_xe_dap',
               'Nha_sach_online','Do_choi','Giat_giu_cham_soc_nha_cua','Cham_soc_thu_cung','Thoi_trang_tre_em','Others']

In [9]:
url_item

['https://shopee.vn/Th%E1%BB%9Di-Trang-Nam-cat.78',
 'https://shopee.vn/Th%E1%BB%9Di-Trang-N%E1%BB%AF-cat.77',
 'https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%A1i-Ph%E1%BB%A5-Ki%E1%BB%87n-cat.84',
 'https://shopee.vn/M%E1%BA%B9-B%C3%A9-cat.163',
 'https://shopee.vn/Thi%E1%BA%BFt-B%E1%BB%8B-%C4%90i%E1%BB%87n-T%E1%BB%AD-cat.2365',
 'https://shopee.vn/Nh%C3%A0-C%E1%BB%ADa-%C4%90%E1%BB%9Di-S%E1%BB%91ng-cat.87',
 'https://shopee.vn/M%C3%A1y-t%C3%ADnh-Laptop-cat.13030',
 'https://shopee.vn/S%E1%BB%A9c-Kh%E1%BB%8Fe-S%E1%BA%AFc-%C4%90%E1%BA%B9p-cat.160',
 'https://shopee.vn/M%C3%A1y-%E1%BA%A3nh-M%C3%A1y-quay-phim-cat.13033',
 'https://shopee.vn/Gi%C3%A0y-D%C3%A9p-N%E1%BB%AF-cat.161',
 'https://shopee.vn/%C4%90%E1%BB%93ng-H%E1%BB%93-cat.9607',
 'https://shopee.vn/T%C3%BAi-V%C3%AD-cat.162',
 'https://shopee.vn/Gi%C3%A0y-D%C3%A9p-Nam-cat.2429',
 'https://shopee.vn/Ph%E1%BB%A5-Ki%E1%BB%87n-Th%E1%BB%9Di-Trang-cat.80',
 'https://shopee.vn/Thi%E1%BA%BFt-B%E1%BB%8B-%C4%90i%E1%BB%87n-Gia-D%E1%BB%A5ng-c

In [10]:
# for category in url_item:
info_all = []
driver.get(url_item[index])
driver = scroll_down_and_wait(driver)
while True:
    current_url= driver.current_url
    print(current_url)
    driver = scroll_down_and_wait(driver)
    time.sleep(8)
    driver, all_urls = get_link_product(driver)
    print('Shopee returned data:', len(all_urls))
    for product_url in all_urls:
        driver, info_page = parse_info_product(driver, product_url, label_marker[index])
        info_all.append(info_page)
            
    print('crawled:', len(info_all))
    driver.get(current_url)
    driver = scroll_down(driver)
    driver.find_element_by_xpath("//button[@class='shopee-icon-button shopee-icon-button--right ']").click()
    if len(info_all)>701:
        break

https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824
Shopee returned data: 45
Confirmed 18+
crawled: 45
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=1
Shopee returned data: 45
crawled: 90
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=2
Shopee returned data: 15
crawled: 105
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=3
Shopee returned data: 50
not found info shop
crawled: 155
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=4
Shopee returned data: 50
crawled: 205
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=5
Shopee returned data: 45
crawled: 250
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=6
Shopee returned data: 45
crawled: 295
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=7
Shopee returned data: 45
crawled: 340
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=8
Shopee returned data: 50
crawled: 390
https://shopee.vn/B%C3%A1ch-Ho%C3%A1-Online-cat.9824?page=9
Shopee returned data: 45
crawled:

In [11]:
len(info_all)

705

In [12]:
df = pd.DataFrame(info_all)
df.head()

,Tên sp:,label:,Danh Mục,Thương hiệu,Xuất xứ,Hạn Sử Dụng,Kho hàng,Gửi từ,sale_price,org_price,...,star,rating,img_url,selled,Đánh Giá,Sản Phẩm,Tỉ Lệ Phản Hồi,Thời Gian Phản Hồi,Tham Gia,Người Theo Dõi
0,Combo 2 Hộp Tiết Kiệm - Yến chưng đường phèn S...,Bach_hoa_online,Shopee-Bách Hoá Online-Quà biếu,Yến sào Khánh Hòa,Việt Nam,16-07-2020,1000,"Quận 12, TP. Hồ Chí Minh",₫540.000,₫540.000,...,0,0,"""https://cf.shopee.vn/file/441ac9e036452604a6e...",0,4,13,100%,trong vài giờ,20 tháng trước,"2,4k"
1,"[🤝𝐐𝐔À 𝐓𝐑𝐈 Â𝐍] 𝐂À 𝐏𝐇Ê 𝐀𝐑𝐀𝐁𝐈𝐂𝐀 𝐁Ộ𝐓, 𝐗𝐀𝐘 𝐏𝐇𝐀 𝐏𝐇𝐈𝐍...",Bach_hoa_online,Shopee-Bách Hoá Online-Đồ uống,No Brand,Việt Nam,NaN,1439,"Thành Phố Long Xuyên, An Giang",₫36.000 - ₫144.000,₫45.000 - ₫160.000,...,5.0,14,"""https://cf.shopee.vn/file/bf47d184bb17aa698c8...",150,49,4,64%,trong vài giờ,6 tháng trước,"9,1k"
2,Combo 12 Hộp Tiết Kiệm - Yến Sào Salanest chưn...,Bach_hoa_online,Shopee-Bách Hoá Online-Quà biếu,Yến sào Khánh Hòa,Việt Nam,28-07-2020,1000,"Quận 12, TP. Hồ Chí Minh",₫3.300.000,₫3.300.000,...,0,0,"""https://cf.shopee.vn/file/b7fc42084935b6e148f...",0,4,13,100%,trong vài giờ,20 tháng trước,"2,4k"
3,[CỰC HOT] 🌶🌶🌶COMBO 2 CHAI VUA NƯỚC CHẤM SIÊU C...,Bach_hoa_online,Shopee-Bách Hoá Online-Đặc sản Việt,No Brand,Việt Nam,30-03-2021,5000,"Quận Gò Vấp, TP. Hồ Chí Minh",₫100.000,₫100.000,...,5.0,8,"""https://cf.shopee.vn/file/cad7762de4d967c4f65...",300,13,3,100%,trong vài giờ,42 ngày trước,"9,1k"
4,[Mã GROMSPS18 hoàn 8% đơn 199K] SẴN HÀNG - Ngũ...,Bach_hoa_online,Shopee-Bách Hoá Online-Đậu & Hạt,No Brand,Trung Quốc,30-06-2021,3094,"Thị Xã Sơn Tây, Hà Nội",₫45.000 - ₫72.000,₫90.000 - ₫130.000,...,4.8,"1,6k",,"7,1k","28,2k",687,97%,trong vài phút,4 năm trước,"27,8k"


In [13]:
df = df.drop_duplicates()

In [14]:
print(df.shape)
df.to_csv('{}.csv'.format(filename), index=False)

(656, 21)
